### Fact Check Assistant: Deploy Strands Agent


## 1. Setup and Installation

First, let's install the required libraries:


In [ ]:
%pip install --upgrade -q -r ../src/frontend/requirements.txt

## 2. Import Libraries and Configure Logging


In [3]:
import time
import logging
import sys
from strands.tools.mcp import MCPClient
from mcp import stdio_client, StdioServerParameters

from strands import Agent, tool
from strands_tools import calculator

from dotenv import load_dotenv
import os

In [ ]:
# Specify the path to your .env file
env_path = os.path.abspath(os.path.join(os.getcwd(), "..", "src", "frontend", ".env"))

# Load the environment variables from the specified file
load_dotenv(dotenv_path=env_path)

## 3. Write Agent for Mathematical Calculations to Frontend


In [ ]:
%%writefile ../src/frontend/math_assistant.py

from strands import Agent, tool
from strands_tools import calculator

MATH_ASSISTANT_SYSTEM_PROMPT = """
You are math wizard, a specialized mathematics applying mathematics across a wide variety of domains.

People need your opinion on the validity of statements they pass to you. Use the tools at your disposal to logically break down a statement to determine whether the statement is true or false. 

Check the statement for mathematical and process errors.

Your capabilities include:
1. Mathematical Operations:
   - Arithmetic calculations
   - Algebraic problem-solving
   - Geometric analysis
   - Statistical computations

2. Teaching Tools:
   - Step-by-step problem solving
   - Visual explanation creation
   - Formula application guidance
   - Concept breakdown

3. Educational Approach:
   - Show detailed work
   - Explain mathematical reasoning
   - Provide alternative solutions
   - Link concepts to real-world applications

4. Australian Taxation Rules
   - Detailed knowledge of Australian Taxation Office Rules (https://www.ato.gov.au/)
   - Ability to perform income tax calculations
   - Ability to perform capital gains tax calculations
   - Analyse structure of personal finances to minimise tax

5. Mortgage Calculations
   - Detailed understanding of how mortgages work in Australia

Focus on clarity and systematic problem-solving while ensuring people understand the underlying concepts.
"""


@tool
def math_assistant(query: str) -> str:
    """
    Process and respond to math-related queries using a specialized math agent.

    Args:
        query: A mathematical question or problem from the user

    Returns:
        A detailed mathematical answer with explanations and steps
    """
    # Format the query for the math agent with clear instructions
    formatted_query = f"Please solve the following mathematical problem, showing all steps and explaining concepts clearly: {query}"

    try:
        # Create the math agent with calculator capability
        math_agent = Agent(
            model="anthropic.claude-3-5-haiku-20241022-v1:0",
            tools=[calculator],
            system_prompt=MATH_ASSISTANT_SYSTEM_PROMPT,
        )

        agent_response = math_agent(formatted_query)
        text_response = str(agent_response)

        if len(text_response) > 0:
            return text_response

        return "I apologize, but I couldn't solve this mathematical problem. Please check if your query is clearly stated or try rephrasing it."
    except Exception as e:
        # Return specific error message for math processing
        return f"Error processing your mathematical query: {str(e)}"


## 4. Write Agent for Researching Erroneous Claims to Frontend


In [ ]:
%%writefile  ../src/frontend/research_assistant.py

from mcp import StdioServerParameters, stdio_client
from strands import Agent, tool
from strands.tools.mcp import MCPClient

RESEARCH_ASSISTANT_SYSTEM_PROMPT = """
You are an assistant specialized in identifying erroneous or misleading claims in text.

Instructions:
- Break down the input text into separate statements.
- For each statement, determine if it contains misinformation, disinformation, or is otherwise clearly false or misleading.
- Fact-check statements using general knowledge and logical reasoning. Look for fabrication, manipulation, or critical omissions.

Output:
- Return a numbered list of all statements you identify as erroneous, incorrect, or misleading.
- Each listed item should be the erroneous claim as a direct quotation.
- If there are no erroneous statements, return 'No erroneous claims found.'
- Do not return any explanations, extra text, or formatting.

Example 1
Input:
Climate change is a hoax invented by scientists. The Great Wall of China is visible from space.

Output:
1. "Climate change is a hoax invented by scientists."
2. "The Great Wall of China is visible from space."

Example 2
Input:
The Pacific Ocean is the largest ocean on Earth. Drinking bleach can cure illnesses.

Output:
1. "Drinking bleach can cure illnesses."

Example 3
Input:

Output:
No erroneous claims found.
"""

stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx",
            args=["duckduckgo-mcp-server"],
        )
    )
)


@tool
def research_assistant(query: str) -> str:
    """
    Fact-check a statement and return whether it's true or false with supporting context.

    Args:
        statement: The statement or claim to fact-check

    Returns:
        A fact-check assessment with verdict (TRUE/FALSE/PARTIALLY TRUE) and supporting evidence
    """
    formatted_query = f"Please fact-check this statement and provide a clear verdict (TRUE/FALSE/PARTIALLY TRUE) with supporting evidence and context: {query}"

    try:
        print("Routed to Fact Checker")
        with stdio_mcp_client:
            # Get the tools from the MCP server
            tools = stdio_mcp_client.list_tools_sync()

            # Create an agent with MCP tools and fact-checking prompt
            fact_checker_agent = Agent(
                model="anthropic.claude-3-5-haiku-20241022-v1:0",
                tools=tools,
                system_prompt=RESEARCH_ASSISTANT_SYSTEM_PROMPT,
            )
            agent_response = fact_checker_agent(formatted_query)
            text_response = str(agent_response)

            if len(text_response) > 0:
                return text_response

            return "I apologize, but I couldn't fact-check this statement. Please provide a clear, specific claim to verify."
    except Exception as e:
        return f"Error during fact-checking: {str(e)}"

## 5. Write Agent for Detecting Erroneous Claims to Frontend


In [ ]:
%%writefile ../src/frontend/claim_detection_assistant.py

from strands import Agent, tool

CLAIM_DETECTION_ASSISTANT_SYSTEM_PROMPT = """
You are an assistant specialized in identifying erroneous or misleading claims in text.

Instructions:
- Break down the input text into separate statements.
- For each statement, determine if it contains misinformation, disinformation, or is otherwise clearly false or misleading.
- Fact-check statements using general knowledge and logical reasoning. Look for fabrication, manipulation, or critical omissions.

Output:
- Return a numbered list of all statements you identify as erroneous, incorrect, or misleading.
- Each listed item should be the erroneous claim as a direct quotation.
- If there are no erroneous statements, return 'No erroneous claims found.'
- Do not return any explanations, extra text, or formatting.

Example 1
Input:
Climate change is a hoax invented by scientists. The Great Wall of China is visible from space.

Output:
1. "Climate change is a hoax invented by scientists."
2. "The Great Wall of China is visible from space."

Example 2
Input:
The Pacific Ocean is the largest ocean on Earth. Drinking bleach can cure illnesses.

Output:
1. "Drinking bleach can cure illnesses."

Example 3
Input:

Output:
No erroneous claims found.
"""


@tool
def claim_detection_assistant(query: str) -> str:
    """
    Analyze free text, segment it into statements, and flag each for erroneous claim.

    Args:
        query: Free-form text containing one or more statements

    Returns:
        The potentially erroneous claims.
    """
    formatted_query = f"Please identify erroneous or misleading claims with supporting evidence and context: {query}"

    try:
        research_agent = Agent(
            model="anthropic.claude-3-5-haiku-20241022-v1:0",
            system_prompt=CLAIM_DETECTION_ASSISTANT_SYSTEM_PROMPT,
        )

        agent_response = research_agent(formatted_query)
        text_response = str(agent_response)

        if len(text_response) > 0:
            return text_response

        return "I apologize, but I couldn't identify any erroneous claims. Please check if your query is clearly stated or try rephrasing it."
    except Exception as e:
        # Return specific error message for math processing
        return f"Error identifying erroneous claims: {str(e)}"

## 6. Write Supervisor Agent for Fact Checking to Frontend


In [ ]:
%%writefile ../src/frontend/fact_check_supervisor.py

import base64
import logging
import os
import sys

from dotenv import load_dotenv

# Import assistant modules from the same directory
from mcp import StdioServerParameters, stdio_client
from strands import Agent, tool
from strands.tools.mcp import MCPClient

# Get the absolute path to the frontend directory (where this script is located)
frontend_dir = os.path.dirname(os.path.abspath(__file__))

# Add the frontend directory to the Python path
if frontend_dir not in sys.path:
    sys.path.append(frontend_dir)

from claim_detection_assistant import claim_detection_assistant
from math_assistant import math_assistant
from research_assistant import research_assistant

SUPERVISOR_SYSTEM_PROMPT = """
You are a sophisticated misinformation detection orchestrator that coordinates specialized agents for comprehensive content analysis and fact-checking. Your primary mission is to detect, analyze, and verify information for potential misinformation.

## Available Specialized Agents

You have access to these specialized agents:

1. **claim_detection_assistant** - Claim Detection Assistant
   - Breaks down input text into separate statements
   - Identifies potentially false or misleading claims
   - Returns a numbered list of erroneous claims as direct quotations
   - Use for: Initial content analysis, statement extraction, identifying suspicious claims

2. **research_assistant** - External Fact Checker Assistant
   - Performs deep fact-checking with evidence using DuckDuckGo search
   - Searches reliable sources and cross-references information
   - Provides TRUE/FALSE/PARTIALLY TRUE verdicts with supporting evidence
   - Use for: Verifying specific claims, finding authoritative sources, detailed fact-checking

3. **math_assistant** - Math Wizard
   - Validates mathematical claims and calculations
   - Checks numerical accuracy and statistical validity
   - Analyzes quantitative statements for errors
   - Use for: Mathematical fact-checking, statistical claims, numerical verification

## Orchestration Strategy

For comprehensive misinformation detection, follow this multi-step approach:

### Step 1: Initial Analysis
- Use **claim_detection_assistant** to segment content and identify potentially problematic statements
- This provides a structured list of all claims that need verification

### Step 2: Specialized Verification
- For mathematical/statistical claims → Use **math_assistant**
- For factual claims requiring evidence → Use **research_assistant**
- For complex claims → Coordinate multiple agents as needed

### Step 3: Synthesis
- Combine results from all agents
- Provide concise assessment with evidence
- Highlight the most concerning misinformation risks

## Trigger Keywords for Misinformation Analysis

Always initiate the multi-agent workflow when you detect:
- "fact-check", "verify claims", "check if true"
- "misinformation", "disinformation", "fake news", "suspicious content"
- Any request to evaluate content truthfulness

## Best Practices

1. **Always start with claim_detection_assistant** for content segmentation
2. **Use research_assistant** for claims requiring external verification
3. **Use math_assistant** for any numerical or statistical claims
4. **Coordinate multiple agents** for comprehensive analysis
5. **Provide clear, evidence-based conclusions**

Your goal is to provide concise, accurate misinformation detection by leveraging the specialized capabilities of each agent in a coordinated manner.
"""

# --- Langfuse & OTEL Observability Setup ---

# Specify the path to your .env file (in the same directory as this script)
env_path = os.path.join(frontend_dir, ".env")

# Load the environment variables from the specified file
load_dotenv(dotenv_path=env_path)

# Configure langfuse for observability
LANGFUSE_AUTH = base64.b64encode(
    f"{os.environ.get('LANGFUSE_PUBLIC_KEY')}:{os.environ.get('LANGFUSE_SECRET_KEY')}".encode()
).decode()

# Handle case where LANGFUSE_HOST might be None
langfuse_host = os.environ.get("LANGFUSE_HOST")
if langfuse_host:
    os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = (
        langfuse_host + "/api/public/otel/v1/traces"
    )
else:
    logging.warning(
        "LANGFUSE_HOST environment variable not set, skipping OTEL configuration"
    )

os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

# Start MCP client
stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx",
            args=["duckduckgo-mcp-server"],
        )
    )
)


## Create Supervisor Agent for Fact Checking
@tool
def fact_check_supervisor(query: str) -> str:
    """
    Process queries through the multi-agent fact-checking system.

    Args:
        query: The text to analyze for factual accuracy

    Returns:
        str: A comprehensive fact-check assessment
    """
    # Use MCP client context manager to ensure session is active
    with stdio_mcp_client:
        try:
            # Combine custom tools with MCP tools
            all_tools = [
                math_assistant,
                claim_detection_assistant,
                research_assistant,
            ]

            # Create the supervisor agent within MCP context
            supervisor = Agent(
                model="anthropic.claude-3-5-haiku-20241022-v1:0",
                system_prompt=SUPERVISOR_SYSTEM_PROMPT,
                tools=all_tools,
            )

            # Process the query within the same MCP context
            return supervisor(
                f"Provide a comprehensive fact-check assessment for this text: {query}"
            )

        except Exception as e:
            logging.error(f"Error processing query with MCP tools: {str(e)}")

            # Fallback to supervisor without MCP tools
            supervisor = Agent(
                model="anthropic.claude-3-5-haiku-20241022-v1:0",
                system_prompt=SUPERVISOR_SYSTEM_PROMPT,
                tools=[
                    math_assistant,
                    claim_detection_assistant,
                ],
            )

            return supervisor(
                f"Provide a comprehensive fact-check assessment for this text: {query}"
            )


## 7. Test the Complete Multi-Agent System

Now let's test the complete multi-agent system.


In [ ]:
import os
import sys

# Add the project root to the path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if project_root not in sys.path:
    sys.path.append(project_root)

# Now import using absolute import
from Fact_Check_Assistant.src.frontend.fact_check_supervisor import (
    fact_check_supervisor,
)

print("\n\nInvoking fact-checking supervisor agent...\n\n")

response = fact_check_supervisor(
    "The Earth is flat and this has been proven by multiple scientific studies."
)

In [ ]:
print(response.message["content"][0]["text"])

## 8. Access the Application in Development Mode

To run the Fact Check Assistant application locally in development mode:

1. **Open a new terminal window** in Jupyter Hub

2. **Navigate to the frontend directory:**

```bash
cd sample-building-agentic-ai-applications-on-aws-fact-check-assistant/Fact_Check_Assistant/src/frontend
```
   
Start the Streamlit application:

```bash
python -m streamlit run app.py
```

Access the application in your browser at:

```https://<session>.studio.<region>.sagemaker.aws/jupyterlab/default/proxy/8501/
```

> ℹ️ Information: Replace <session> and <region> with the actual values from your current browser URL. You can find these values in your SageMaker Studio address bar.

The application will be available on port 8501 and accessible through the SageMaker Studio proxy.

## [Optional] 9. Deploy the CDK Stack for the Fact Checking Application

> Information: ℹ️ You will need to execute steps 9 and 10 outside of SageMaker Studio. The environment you use must meet the flow **prerequisites**.

This AWS CDK stack sets up a containerized application that to run a streamlit application that interacts with our fact checking agent created above. It automatically builds and deploys the service using AWS Fargate (a serverless container platform) and exposes it through a load-balanced endpoint.

> ⚠️ Warning: The **Application Load Balancer (ALB)** created by this stack is **publicly accessible** over the internet. This means: **Anyone with the ALB DNS name can send requests** to your service (assuming security group and app-level controls allow it). While this is necessary for public-facing applications, it can pose a **security risk** if not properly protected.

Prerequisites

- [AWS CLI](https://aws.amazon.com/cli/) installed and configured
- [Node.js](https://nodejs.org/) (v18.x or later)
- Python 3.11 or later
  - Either:
    - [Podman](https://podman.io/) installed and running
    - (or) [Docker](https://www.docker.com/) installed and running
    - Ensure podman or docker daemon is running.


In [ ]:
!bash ../src/infra/deploy.sh

# [Optional] 10. Access the application

The application can be opened in a web browser by navigating to the URL provided in the output of the CDK deployment script. This URL is the endpoint for the load-balanced application. You can also access the application by using the ALB DNS name provided in the output of the CDK deployment script.

> ⚠️ Warning: The **Application Load Balancer (ALB)** created by this stack is **publicly accessible** over the internet. This means: **Anyone with the ALB DNS name can send requests** to your service (assuming security group and app-level controls allow it). While this is necessary for public-facing applications, it can pose a **security risk** if not properly protected.


In [ ]:
import subprocess

# Get the service URL from CDK output using AWS CLI
result = subprocess.run(
    [
        "aws",
        "cloudformation",
        "describe-stacks",
        "--stack-name",
        "FactCheckAssistantStack",
        "--query",
        "Stacks[0].Outputs[?OutputKey=='ApplicationURL'].OutputValue",
        "--output",
        "text",
    ],
    capture_output=True,
    text=True,
)

SERVICE_URL = result.stdout.strip()
print(f"Service URL: {SERVICE_URL}")